In [108]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris 
import seaborn as sns
from utils import variables as var
from sklearn.linear_model import LinearRegression
#Financiero
import yfinance as yf
import quantstats as qs
import ta

In [109]:
# We mainly uses the csv archives + yfinance as an auxiliary module
gold=pd.read_csv('data/Gold_data.csv')
index_sp500=pd.read_csv('data/sp500_index.csv')
company=pd.read_csv('data/sp500_companies.csv')
stocks=pd.read_csv(r"C:\Users\Adrian\Documents\DATA_SCIENCE_BOOTCAMP\DATA\sp500_stocks.csv")
buffet=stocks[stocks['Symbol']=='BRK-B']
buffet
magnificas=stocks[stocks['Symbol'].isin(['MSFT','AAPL','META','GOOG','AMZN'])]
waste=stocks[stocks['Symbol']=='WM']
waste.head()


,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
1741432,2010-01-04,WM,23.382090,34.160000,34.259998,33.869999,34.009998,2058800.0
1741433,2010-01-05,WM,23.279421,34.009998,34.220001,33.779999,34.220001,2961700.0
1741434,2010-01-06,WM,23.272572,34.000000,34.150002,33.779999,33.869999,1636700.0
1741435,2010-01-07,WM,23.327345,34.080002,34.119999,33.820000,33.939999,1406700.0
1741436,2010-01-08,WM,23.436859,34.240002,34.330002,33.669998,33.919998,1451900.0


## ESTADO Y LIMPIEZA DE LOS DATOS

In [110]:
#VAMOS A MIRAR EL ESTADO DE LOS DATOS DE CADA COLUMNA
gold.info()
print('//////////////////////////////////////////////////// \n')
index_sp500.info()
print('//////////////////////////////////////////////////// \n')
company.info()
print('//////////////////////////////////////////////////// \n')
stocks.info()
print('//////////////////////////////////////////////////// \n')
buffet.info()
print('//////////////////////////////////////////////////// \n')
magnificas.info()
print('//////////////////////////////////////////////////// \n')
waste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5929 entries, 0 to 5928
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    5929 non-null   object 
 1   open    5929 non-null   float64
 2   high    5929 non-null   float64
 3   low     5929 non-null   float64
 4   close   5929 non-null   float64
 5   volume  5929 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 278.1+ KB
//////////////////////////////////////////////////// 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518 entries, 0 to 2517
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2518 non-null   object 
 1   S&P500  2518 non-null   float64
dtypes: float64(1), object(1)
memory usage: 39.5+ KB
//////////////////////////////////////////////////// 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 16 columns):
 #   Column 

- PARA NUESTRO ESTUDIO NO NECESITAMOS PRECIO APERTURA, MAXIMO DIARIO Y MÍNIMO DIARIO: SE DROPEARAN ESAS COLUMNAS
- NECESITAMOS GENERAR UNA COLUMNA CON LA FECHA EN FORMATO DATETIME YA QUE NOS VIENE COMO OBJECT
- EN EL DATASET COMPANIES LOS VALORES NULOS SON CASI INAPRECIABLES COMO PARA DESVIAR EL ESTUDIO

#### OJO 
LOS VALORES NULOS QUE TENEMOS HACEN REFERENCIA A LOS DÍAS FESTIVOS, CUYO VOLUMEN DE NEGOCIACION ES 0 Y EL PRECIO NO VARÍA EN NINGUN MOMENTO, CON LO QUE HAREMOS UN DROP DE ESAS FILAS

### ELIMINADO VALORES FESTIVOS

In [111]:
stocks=stocks.dropna()
stocks.info()
buffet = buffet.dropna()
magnificas = magnificas.dropna()
waste=waste.dropna()
waste.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1722715 entries, 0 to 1809793
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Date       object 
 1   Symbol     object 
 2   Adj Close  float64
 3   Close      float64
 4   High       float64
 5   Low        float64
 6   Open       float64
 7   Volume     float64
dtypes: float64(6), object(2)
memory usage: 118.3+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 3598 entries, 1741432 to 1745029
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       3598 non-null   object 
 1   Symbol     3598 non-null   object 
 2   Adj Close  3598 non-null   float64
 3   Close      3598 non-null   float64
 4   High       3598 non-null   float64
 5   Low        3598 non-null   float64
 6   Open       3598 non-null   float64
 7   Volume     3598 non-null   float64
dtypes: float64(6), object(2)
memory usage: 253.0+ KB


### ELIMINADO VALORES DE TRADING

In [112]:
lista_columnas=['open','high','low']
lista_columnas1=['Open','High','Low']

gold=gold.drop(lista_columnas, axis=1)
gold.info()
stocks=stocks.drop(lista_columnas1,axis=1)
buffet=buffet.drop(lista_columnas1,axis=1)
magnificas=magnificas.drop(lista_columnas1,axis=1)
waste=waste.drop(lista_columnas1,axis=1)
waste.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5929 entries, 0 to 5928
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    5929 non-null   object 
 1   close   5929 non-null   float64
 2   volume  5929 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 139.1+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 3598 entries, 1741432 to 1745029
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       3598 non-null   object 
 1   Symbol     3598 non-null   object 
 2   Adj Close  3598 non-null   float64
 3   Close      3598 non-null   float64
 4   Volume     3598 non-null   float64
dtypes: float64(3), object(2)
memory usage: 168.7+ KB


### CREACION COLUMNAS DATE TIME Y AÑO

In [113]:
#A PARTIR DEL DATE OBJECT LO PASAMOS A DATETIME Y GENERAMOS LA COLUMNA DATETIME
gold['DateTime']=pd.to_datetime(gold['date'])
#CON EL DATE TIME GENERAMOS LA COLUMNA YEAR
gold['year']=gold['DateTime'].dt.year

gold.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5929 entries, 0 to 5928
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      5929 non-null   object        
 1   close     5929 non-null   float64       
 2   volume    5929 non-null   int64         
 3   DateTime  5929 non-null   datetime64[ns]
 4   year      5929 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1), object(1)
memory usage: 208.6+ KB


In [114]:
index_sp500['DateTime']=pd.to_datetime(index_sp500['Date'])
index_sp500['year']=index_sp500['DateTime'].dt.year
index_sp500.info()

stocks['DateTime']=pd.to_datetime(stocks['Date'])
stocks['year']=stocks['DateTime'].dt.year

buffet['DateTime']=pd.to_datetime(buffet['Date'])
index_sp500['year']=index_sp500['DateTime'].dt.year

magnificas['DateTime']=pd.to_datetime(magnificas['Date'])
magnificas['year']=magnificas['DateTime'].dt.year

waste['DateTime']=pd.to_datetime(waste['Date'])
waste['year']=waste['DateTime'].dt.year
waste.head()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518 entries, 0 to 2517
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      2518 non-null   object        
 1   S&P500    2518 non-null   float64       
 2   DateTime  2518 non-null   datetime64[ns]
 3   year      2518 non-null   int32         
dtypes: datetime64[ns](1), float64(1), int32(1), object(1)
memory usage: 69.0+ KB


,Date,Symbol,Adj Close,Close,Volume,DateTime,year
1741432,2010-01-04,WM,23.382090,34.160000,2058800.0,2010-01-04,2010
1741433,2010-01-05,WM,23.279421,34.009998,2961700.0,2010-01-05,2010
1741434,2010-01-06,WM,23.272572,34.000000,1636700.0,2010-01-06,2010
1741435,2010-01-07,WM,23.327345,34.080002,1406700.0,2010-01-07,2010
1741436,2010-01-08,WM,23.436859,34.240002,1451900.0,2010-01-08,2010


## ANALISIS PRIMERA Y SEGUNDA HIPÓTESIS:

### EL SP500 ES EL MEJOR INDICE PARA INVERTIR SEGUIDO DEL ORO